<a href="https://colab.research.google.com/github/rky0930/style_transfer_ai4teen/blob/master/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EC%9C%BC%EB%A1%9C_%EC%98%88%EC%88%A0_%EC%9E%91%ED%92%88_%EB%A7%8C%EB%93%A4%EA%B8%B0_%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인공지능으로 예술 작품 만들기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/models/blob/master/research/nst_blogpost/4_Neural_Style_Transfer_with_Eager_Execution.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

이 튜토리얼은 [Neural Style Transfer with tf.keras](#)을 기반으로 제작 되었습니다.  
내용을 한글화하고 인공지능 기술 직접 체험하는 것에 초점을 맞춰 기술적인 자세한 내용은 배제하고 제작하였습니다.

## 개요
이 튜토리얼에서는 인공지능 분야에서 가장 활발하게 사용되는 딥러닝 기술을 사용하여  두 사진의 "컨텐츠"와 "스타일"을 합성하는 방법을 배울 것 입니다.
이 기술은 "Neural Style Transfer" 불립니다. 자세한 설명은 이 논문([Leon A. Gatys' paper, A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576)) 을 참조하시 기 바랍니다. **단, 이 튜토리얼을 완수하기 위해 논문을 읽을 필요는 없습니다 !**

그래서,  Neural Style Transfer 란 무엇일까요 ?

Neural Style Transfer는 2장의 사진에서 각각 "컨텐츠"와 "스타일"을 추출한 후 하나의 사진으로 만드는 기술 입니다.  
 = 컨텐츠 사진 +  스타일 사진  = 컨텐츠 + 스타일 사진  
 = 거북이 사진 + 유명 화가의 그림 = 유명 화가의 스타일로 그린 거북이 사진    

예를들어, 아래 두 사진을 보겠습니다. 

<img src="https://github.com/rky0930/style_transfer_ai4teen/blob/master/images/gwanghwamun.jpg?raw=1" alt="Drawing" style="width: 200px;"/>  
광화문 *출처 한국관광공사 홈페이지([광화문](http://tong.visitkorea.or.kr/cms/resource/84/2526484_image2_1.jpg))
  
<img src="https://github.com/rky0930/style_transfer_ai4teen/blob/master/images/leejungseob_white_ox.jpg?raw=1" alt="Drawing" style="width: 200px;"/>  
이중섭 - 흰 소

이제 이중섭 그림 흰 소의 스타일을 학습하여 "광화문"을 그리면..  
<img src="https://github.com/rky0930/style_transfer_ai4teen/blob/master/images/leejungseob_gwangwhamoon.png?raw=1" alt="Drawing" style="width: 200px;"/>

어떤가요 ? 정말 이중섭 작가가 광화문 그림 같나요 ? 마법같나요 ? 어떤한 마법도 사용하지 않았습니다. 사실, Style Transfer 기술은 인공지능을 만들때 사용되는 인공 신경망(Neural Network) 내부의 특정 부분을 시각화 한 것입니다. 시각화 결과는 내부가 어떻게 동작하는지 간접적으로 보여주기 때문에 합성된 결과물 뿐만아니라 그 의미 또한 재밌는 기술 입니다.   

### Style Transfer 진행 순서

1. 데이터 시각화
2. 기본적인 데이터 가공 
3. 인공 신경망 모델 만들기
4. Loss 함수 설정
5. 딥러닝 모델 훈련
6. 결과 확인

** 읽으실 분들 ** 이 자료는 기본적인 머신러닝 지식 없이 인공지능 기술을 직접 사용해 볼 수 있도록 제작 되었습니다. 이 자료를 읽은 많은 분들이 인공지능 기술에 흥미를 가지게 되고, 인공지능 관련 공부를 시작하게 되는 계기가 되기를 바랍니다.   
더 많은 내용을 알고 싶은 분들은 아래 자료를 참고해 주세요.   
* [Neural Style Transfer with tf.keras](#)
* [Gatys' paper](https://arxiv.org/abs/1508.06576) 
* [Understand reducing loss with gradient descent](https://developers.google.com/machine-learning/crash-course/reducing-loss/gradient-descent)

**튜토리얼 완수 예상 소요 시간**: 15 분 


## 설정하기

### 사진 다운로드

In [0]:
import os
img_dir = '/tmp/nst'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
!wget --quiet -P /tmp/nst/ https://github.com/rky0930/style_transfer_ai4teen/blob/master/images/gwanghwamun.jpg?raw=1
!wget --quiet -P /tmp/nst/ https://github.com/rky0930/style_transfer_ai4teen/blob/master/images/leejungseob_white_ox.jpg?raw=1
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/d/d7/Green_Sea_Turtle_grazing_seagrass.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/0a/The_Great_Wave_off_Kanagawa.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/b/b4/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/0/00/Tuebingen_Neckarfront.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/6/68/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg
!wget --quiet -P /tmp/nst/ https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg


In [0]:
!ls /tmp/nst/*

### 모듈 가져온후 설정

In [0]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models 
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as K
tf.__version__
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

## 입력 사진 시각화

In [0]:
# Set up some global values here
content_path = '/tmp/nst/gwanghwamun.jpg?raw=1'
style_path = '/tmp/nst/leejungseob_white_ox.jpg?raw=1'

In [0]:
def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)
  return img

In [0]:
def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

컨턴츠 사진과 스타일 사진 확인하기.  
우리가 원하는 결과는 컨텐츠 사진을 스타일 사진의 스타일로 합성하는 것 입니다 !

In [0]:
plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()

## 기본적인 데이터 가공
### 데이터 전처리 & 후처리

In [0]:
def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x


### 사진에서 컨텐츠와 스타일 얻는 방법
우리는 사진에서 "컨텐츠"와 "스타일"을 얻기위해서 딥러닝 모델을 사용할 것 입니다. 우리가 사용할 딥러닝 모델은 여러층으로 이루어져 있는데, 뒤쪽에 위치한 깊은 층일 수록 입력한 사진의 고차원 적인 특징이 추출되고, 앞쪽의 앝은층 일수록 단순한 특징이 추출되는 특성이 있습니다. 이 튜토리얼에서 사용할 딥러닝 모델의 이름은 VGG19 입니다. 원래는 사진 분류기에 사용하기 위해 학습 되었지만 입력된 사진의 "컨텐츠"와 "스타일"을 추출 할 때도 사용 할 수 있습니다.

#### 사진 분류기용 신경망으로 "컨텐츠"와 "스타일"을 추출 ?
어떻게 사진 분류기용 딥러닝 모델을 사용하여 "컨텐츠" 와 "스타일"을 추출 하는지 궁금하신가요? 우선, 우리가 사용하는 사진 분류기용 딥러닝 모델이 어떻게 동작하는지 설명해야 합니다. 사진 분류기에서 사용되는 딥러닝 모델은 사진을 픽셀 단위로 입력 받아 딥러닝 내부적인 표현 방식으로 변환하여 입력 받은 사진에 존재하는 복잡한 특징을 자체적으로 이해합니다. 특정 사물의 사진들에 공통적으로 존재하는 특징 찾는다면, 이 특징들을 사용하여 사진 분류기로 동작 시킬 수 있습니다. 예를들어, 강아지 사진에만 존재하는 특징을 찾고, 고양이 사진에만 존재하는 특징을 찾는다면 강아지 사진과 고양이 사진을 분류할 수 있는 사진 분류기를 만들 수 있겠죠 ! 이게 바로 딥러닝 기술의 마법 같은 능력입니다. 딥러닝을 사용하여 사진에 존재하는 특징을 추출 하고 이 특징들을 사진 분류기에 사용하는 것 입니다. 그럼 Style Transfer는 어떻게 동작할까요 ? 사진의 "컨텐츠"와 "스타일" 또한 사진의 특징 중 하나 입니다. 다만,  Style Transfer에서는 사진을 분류하는데 사용하지 않고 추출한 두 특징을 하나의 사진으로 합성하여 새로운 사진을 만들어 내는 것 입니다. 

## 딥러닝 모델 만들기
앞서 말쓴드린 듯이 우리는 [VGG19](https://keras.io/applications/#vgg19)를 사용할 것 입니다. 이 모델은 원래 논문에서 제시되었기도 하고 비교적 간단하면서  높은 성능을 보여줍니다. 이 모델을 사용하여 사진의 "컨텐츠"와 "스타일"을 추출하여 합성된 이미지를 만들 것입니다. 


In [0]:
# Content layer where will pull our feature maps
content_layers = ['block5_conv2'] 

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model():
  """ Creates our model with access to intermediate layers. 
  
  This function will load the VGG19 model and access the intermediate layers. 
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model. 
  
  Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
  """
  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers 
  style_outputs = [vgg.get_layer(name).output for name in style_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = style_outputs + content_outputs
  # Build model 
  return models.Model(vgg.input, model_outputs)

def get_feature_representations(model, content_path, style_path):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
  """
  # Load our images in 
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)
  
  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)
  
  
  # Get the style and content feature representations from our model  
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

방금 우리는 위 소스코드를 실행하여 학습이 완료된 사진분류기용 딥러닝 모델을 로딩했습니다. 그리고, 우리가 사용할 "컨텐츠"와 "스타일" 특징이 저장되어 있는 층을 지정하고 특징 추출에 사용할 함수를 정의하였습니다.


## Loss 함수 와 경사 하강법
이 글을 읽는 초심자들에게 Loss함수, 경사하강법과 역전파 등은 익숙한 개념이 아닐 수 있습니다. 이들은 딥러닝 모델을 학습시키기 위해 필수 적인 아주 중요한 요소입니다. 다만, 이 내용은 수학적 배경이 필요하기 때문에 이 튜토리얼에서는 제외 하였습니다. 이 튜토리얼을 실행하기위해 당장 이해할 필요는 없습니다. 자세한 내용이 궁금하신 분들은 튜토리얼 초반에 언급한 자료를 확인해 주세요.   
아래의 코드를 실행하면  Loss 함수 와 경사하강에 관련된 필요한 파이썬 함수들이 정의 됩니다. 



In [0]:
def get_content_loss(base_content, target):
  return tf.reduce_mean(tf.square(base_content - target))

def gram_matrix(input_tensor):
  # We make the image channels first 
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)

def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
  """This function will compute the loss total loss.
  
  Arguments:
    model: The model that will give us access to the intermediate layers
    loss_weights: The weights of each contribution of each loss function. 
      (style weight, content weight, and total variation weight)
    init_image: Our initial base image. This image is what we are updating with 
      our optimization process. We apply the gradients wrt the loss we are 
      calculating to this image.
    gram_style_features: Precomputed gram matrices corresponding to the 
      defined style layers of interest.
    content_features: Precomputed outputs from defined content layers of 
      interest.
      
  Returns:
    returns the total loss, style loss, content loss, and total variational loss
  """
  style_weight, content_weight = loss_weights
  
  # Feed our init image through our model. This will give us the content and 
  # style representations at our desired layers. Since we're using eager
  # our model is callable just like any other function!
  model_outputs = model(init_image)
  
  style_output_features = model_outputs[:num_style_layers]
  content_output_features = model_outputs[num_style_layers:]
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, style_output_features):
    style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, content_output_features):
    content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
  style_score *= style_weight
  content_score *= content_weight

  # Get total loss
  loss = style_score + content_score 
  return loss, style_score, content_score

def compute_grads(cfg):
  with tf.GradientTape() as tape: 
    all_loss = compute_loss(**cfg)
  # Compute gradients wrt input image
  total_loss = all_loss[0]
  return tape.gradient(total_loss, cfg['init_image']), all_loss


## 입력 사진에 Style Transfer 적용하기


### 딥러닝 훈련 함수 정의
아래의 소스코드를 실행하면 딥러닝 모델을 훈련하기위한 파이썬 함수가 정의 됩니다. 

In [0]:
import IPython.display

def run_style_transfer(content_path, 
                       style_path,
                       num_iterations=1000,
                       content_weight=1e3, 
                       style_weight=1e-2): 
  # We don't need to (or want to) train any layers of our model, so we set their
  # trainable to false. 
  model = get_model() 
  for layer in model.layers:
    layer.trainable = False
  
  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
  # Set initial image
  init_image = load_and_process_img(content_path)
  init_image = tfe.Variable(init_image, dtype=tf.float32)
  # Create our optimizer
  opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)

  # For displaying intermediate images 
  iter_count = 1
  
  # Store our best result
  best_loss, best_img = float('inf'), None
  
  # Create a nice config 
  loss_weights = (style_weight, content_weight)
  cfg = {
      'model': model,
      'loss_weights': loss_weights,
      'init_image': init_image,
      'gram_style_features': gram_style_features,
      'content_features': content_features
  }
    
  # For displaying
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()
  
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means   
  
  imgs = []
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time() 
    
    if loss < best_loss:
      # Update best loss and best image from total loss. 
      best_loss = loss
      best_img = deprocess_img(init_image.numpy())

    if i % display_interval== 0:
      start_time = time.time()
      
      # Use the .numpy() method to get the concrete numpy array
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_img))
      print('Iteration: {}'.format(i))        
      print('Total loss: {:.4e}, ' 
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
  print('Total time: {:.4f}s'.format(time.time() - global_start))
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])
      
  return best_img, best_loss 

### 딥러닝 훈련 함수 실행
아래의 소스코드를 실행하면 딥러닝 모델의 훈련이 시작 됩니다. 약 10정도 소요됩니다. 

In [0]:
best, best_loss = run_style_transfer(content_path, 
                                     style_path, num_iterations=1000)

#### 결과 확인하기

In [0]:
Image.fromarray(best)

## 결과물 확인하기

In [0]:
def show_results(best_img, content_path, style_path, show_large_final=True):
  plt.figure(figsize=(10, 5))
  content = load_img(content_path) 
  style = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style, 'Style Image')

  if show_large_final: 
    plt.figure(figsize=(10, 10))

    plt.imshow(best_img)
    plt.title('Output Image')
    plt.show()

In [0]:
show_results(best, content_path, style_path)

## 다른 사진 변환해 보기

### 반고흐 - 별이 빛나는 밤 + 독일 도시 (튀빙겐)

In [0]:
best_starry_night, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg',
                                                  '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

In [0]:
show_results(best_starry_night, '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/1024px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

### 창조의 기둥 +  독일 도시 (튀빙겐)
창조의 기둥이란 허블 우주망원경이 지구로부터 약 7,000 광년 떨어진 독수리 성운의 성간가스와 성간먼지의 덩어리를 촬영한 사진

In [0]:
best_poc_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [0]:
show_results(best_poc_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

### 구성VII + 독일 도시 (튀빙겐)
구성VII: 바실리 칸딘스키의 그림

In [0]:
best_kandinsky_tubingen, best_loss = run_style_transfer('/tmp/nst/Tuebingen_Neckarfront.jpg', 
                                                  '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

In [0]:
show_results(best_kandinsky_tubingen, 
             '/tmp/nst/Tuebingen_Neckarfront.jpg',
             '/tmp/nst/Vassily_Kandinsky,_1913_-_Composition_7.jpg')

### 창조의 기둥 + 바다 거북
창조의 기둥이란 허블 우주망원경이 지구로부터 약 7,000 광년 떨어진 독수리 성운의 성간가스와 성간먼지의 덩어리를 촬영한 사진


In [0]:
best_poc_turtle, best_loss = run_style_transfer('/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg', 
                                                  '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

In [0]:
show_results(best_poc_turtle, 
             '/tmp/nst/Green_Sea_Turtle_grazing_seagrass.jpg',
             '/tmp/nst/Pillars_of_creation_2014_HST_WFC3-UVIS_full-res_denoised.jpg')

#### 사진 직접 올려보기 

In [0]:
from google.colab import files
my_style_image = files.upload()

In [0]:
my_contents_image = files.upload()

In [0]:
print(list(my_contents_image.keys())[0], list(my_style_image.keys())[0])

In [0]:
best_image, best_loss = run_style_transfer(list(my_contents_image.keys())[0], list(my_style_image.keys())[0])

In [0]:
show_results(best_image, list(my_contents_image.keys())[0], list(my_style_image.keys())[0])

#### 사진 정보 

**[Image of Tuebingen](https://commons.wikimedia.org/wiki/File:Tuebingen_Neckarfront.jpg)** 
Photo By: Andreas Praefcke [GFDL (http://www.gnu.org/copyleft/fdl.html) or CC BY 3.0  (https://creativecommons.org/licenses/by/3.0)], from Wikimedia Commons

**[Image of Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg)**
By P.Lindgren [CC BY-SA 3.0 (https://creativecommons.org/licenses/by-sa/3.0)], from Wikimedia Commons

